In [ ]:
!pip install pandas
!pip install pyspark
!pip install pyarrow

In [ ]:
from pyspark.sql import SparkSession
 
# Building the SparkSession and name 
# it :'pandas to spark'
spark = SparkSession.builder.appName(
  "pandas to spark").getOrCreate()

spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [3]:
import pandas as pd

data = [[1, 'Alice'], [2, 'Bob'], [13, 'John'], [6, 'Alex']]
students = pd.DataFrame(data, columns=['student_id', 'student_name']).astype({'student_id':'Int64', 'student_name':'object'})
data = [['Math'], ['Physics'], ['Programming']]
subjects = pd.DataFrame(data, columns=['subject_name']).astype({'subject_name':'object'})
data = [[1, 'Math'], [1, 'Physics'], [1, 'Programming'], [2, 'Programming'], [1, 'Physics'], [1, 'Math'], [13, 'Math'], [13, 'Programming'], [13, 'Physics'], [2, 'Math'], [1, 'Math']]
examinations = pd.DataFrame(data, columns=['student_id', 'subject_name']).astype({'student_id':'Int64', 'subject_name':'object'})

In [4]:
students = spark.createDataFrame(students)
students.show()

subjects = spark.createDataFrame(subjects)
subjects.show()

examinations = spark.createDataFrame(examinations)
examinations.show()


+----------+------------+
|student_id|student_name|
+----------+------------+
|         1|       Alice|
|         2|         Bob|
|        13|        John|
|         6|        Alex|
+----------+------------+

+------------+
|subject_name|
+------------+
|        Math|
|     Physics|
| Programming|
+------------+

+----------+------------+
|student_id|subject_name|
+----------+------------+
|         1|        Math|
|         1|     Physics|
|         1| Programming|
|         2| Programming|
|         1|     Physics|
|         1|        Math|
|        13|        Math|
|        13| Programming|
|        13|     Physics|
|         2|        Math|
|         1|        Math|
+----------+------------+



In [30]:
from pyspark.sql.functions import count

students\
.join(
    subjects,
    None,
    'cross'
) \
.join(
    examinations \
    .groupby(['student_id', 'subject_name']) \
    .agg(count('subject_name').alias('attended_exams')),
    ['student_id', 'subject_name'],
    'left'
) \
.fillna(0) \
.show()

+----------+------------+------------+--------------+
|student_id|subject_name|student_name|attended_exams|
+----------+------------+------------+--------------+
|         1|        Math|       Alice|             3|
|         1|     Physics|       Alice|             2|
|         1| Programming|       Alice|             1|
|         2|        Math|         Bob|             1|
|         2|     Physics|         Bob|             0|
|         2| Programming|         Bob|             1|
|        13|        Math|        John|             1|
|        13|     Physics|        John|             1|
|        13| Programming|        John|             1|
|         6|        Math|        Alex|             0|
|         6|     Physics|        Alex|             0|
|         6| Programming|        Alex|             0|
+----------+------------+------------+--------------+

